Created by Anna Vorreuther

In [1]:
import pandas as pd
import numpy as np

In [6]:
# load individual participants' data
data = pd.read_csv("../data + material/data/derivative/data.csv", sep=";", index_col=False)

data = data.append(dict(data.mean(axis=0, numeric_only=True)), ignore_index=True)
data = data.append(dict(data.std(axis=0, numeric_only=True)), ignore_index=True)

data = data[:-2] # drop mean and std, not needed here

# determine channel of interest based on highest t-value of localizer runs
channel_vals = data.filter([
                            'best_channel_setupa_hbo_value',
                            'best_channel_setupa_hbr_value',
                            'best_channel_setupb_hbo_value', 
                            'best_channel_setupb_hbr_value'
                        ], axis=1)
coi = channel_vals.max(axis=1)
data["coi_value"] = coi

# get COI names for each participant
max_indices = np.argmax(np.array(channel_vals), axis=1)
channels = data.filter([
    'best_channel_setupa_hbo', 
    'best_channel_setupa_hbr', 
    'best_channel_setupb_hbo', 
    'best_channel_setupb_hbr'
], axis=1)
coi_names = np.array(channels)[np.arange(len(max_indices)), max_indices]
coi_names = [tuple(c.split('-')) for c in list(coi_names)]

C:\Users\Anna\AppData\Local\Temp\ipykernel_77508\3741955463.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dict(data.mean(axis=0, numeric_only=True)), ignore_index=True)
C:\Users\Anna\AppData\Local\Temp\ipykernel_77508\3741955463.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(dict(data.std(axis=0, numeric_only=True)), ignore_index=True)


In [82]:
# List of file paths
paths = [
    "path/Standard_Channels.txt",
    "path2/Standard_Channels.txt"
]

paths = [
    "C:/Users/Anna/Dropbox/Betty & Anna/TEMCOC/Standard_Channels.txt", # checkerboard, corresponds to setup A above
    "C:/Users/Anna/Dropbox/Betty & Anna/TEMCOR/Standard_Channels.txt", # rowed, corresponds to setup B above
        ]

# Short-distance channels to exclude from average distance measure
SDC = [(9, 5)]

coi_dists = list()

# Iterate over each file path
for pidx, p in enumerate(paths):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(p, header=None, sep=",")
    
    # Only take some values from Standard_Channels.txt
    df = df[[0, 1, 2, 6]]
    
    # Assign column names to the DataFrame
    df.columns = ["channelNo", "sources", "detectors", "distances"]
    print(df.to_string(index=False))
    
    # Print the total number of channels in the setup
    print(f"Total channels in the setup: {len(df)}")
    
    # Iterate over each short-distance channel to exclude
    for sdc in SDC:
        # Find rows where sources and detectors match the short-distance channel
        rows_to_exclude = df[(df["sources"] == sdc[0]) & (df["detectors"] == sdc[1])].index

        # Drop the rows to exclude from the DataFrame
        df = df.drop(rows_to_exclude)
    
    # Print the number of channels without the short-distance channels
    print(f"Channels in the setup without short-distance channels: {len(df)}")
    
    # Calculate the mean channel distance according to NIRSite
    mean_distance = df["distances"].mean()
    print(f"Mean channel distance according to NIRSite: {mean_distance}")
    std_distance = df["distances"].std()
    print(f"StD channel distance according to NIRSite: {std_distance}\n")
    
    # Print out distances of COIs for each setup according to NIRSite
    for coi in coi_names:
#        print(coi)
        idx = df[(df["sources"] == int(coi[0])) & (df["detectors"] == int(coi[1]))].index
        coi_dist = df["distances"][idx]
        coi_dists.append((f"{'A' if pidx == 0 else 'B'}",coi_dist.to_string(index=False)))
 #       print(coi_dist.to_string(index=False))

print(coi_dists)
# print out distance of COI for each participant
cois = list()
for idx, c in enumerate(data["setup_choice"]):
    print(idx+1)
    if c == "A":
        coi = coi_dists[idx]
    else:
        coi= coi_dists[idx+10]
    print(coi)
    cois.append(coi)
coisdf = pd.DataFrame.from_records(cois, columns=["setup", "distance"])
# P08 has an incorrect setup choice
print(coisdf)
coisdf.iloc[7]["setup"] = "B"
coisdf.iloc[7]["distance"] = df["distances"][df[(df["sources"] == 8) & (df["detectors"] == 7)].index].to_string(index=False)
print(coisdf)

# print mean and std for all COIs, checkerboard, and rowed setup
print(f'mean: {coisdf["distance"].astype(float).mean()}')
print(f'std: {coisdf["distance"].astype(float).std()}')
print("\nSetup checkerboard")
coia = coisdf[coisdf["setup"] == "A"]
print(f'mean: {coia["distance"].astype(float).mean()}')
print(f'std: {coia["distance"].astype(float).std()}')
print("Setup rowed")
coib = coisdf[coisdf["setup"] == "B"]
print(f'mean: {coib["distance"].astype(float).mean()}')
print(f'std: {coib["distance"].astype(float).std()}')

 channelNo  sources  detectors  distances
         1        1          1       34.1
         2        1          2       34.0
         3        1          3       37.0
         4        2          2       40.7
         5        2          3       37.6
         6        2          4       36.7
         7        2          6       40.9
         8        3          2       33.6
         9        3          4       40.1
        10        4          4       36.4
        11        4          8       33.0
        12        5          3       37.2
        13        5          5       37.0
        14        5          6       39.3
        15        6          5       40.1
        16        6          6       41.0
        17        6          7       39.9
        18        7          4       40.3
        19        7          6       38.9
        20        7          7       40.2
        21        7          8       38.5
        22        8          7       39.7
        23        8          8    